# Read npz to get Pressure and Grid and to save grid (optional)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

in_dir = '/home/jovyan/share/mgn_data/bleve3d/npz/100071.npz'

data = np.load(in_dir)
pressure = data['pressure']
grid = data['coordinate']

print(pressure.shape, grid.shape)

step = 1
pressure = pressure[:100, ::step, ::step, ::step]
grid = grid[::step, ::step, ::step, :]

x_core = np.concatenate((np.arange(0, 49, 2), np.arange(49, 65), np.arange(65, 98, 2)))
y_core = np.concatenate((np.arange(0, 24, 2), np.arange(24, 74), np.arange(74, 98, 2), np.array([97])))
z_core = np.concatenate((np.arange(0, 15, 1), np.arange(15, 49, 2), np.array([48])))                  
    
grid = grid[z_core, :, :, :][:, y_core,:, :][:, :, x_core, :]

x = grid[..., 0].flatten()
y = grid[..., 1].flatten()
z = grid[..., 2].flatten()
print(pressure.shape, grid.shape)
#np.save('/home/jovyan/share/mgn_data/bleve3d/grid-core', grid)

# Compute statistics of 52 data used by BGN

In [ ]:
import pandas as pd
import numpy as np
import h5py
import glob
import random
import os

file_path = '/home/jovyan/work/meshGraphNets_pytorch/data/bleve3d/BLEVE_Open_500Butan_500Propane_5-40m_8outputs.xlsx'
npz_dir = '/home/jovyan/share/mgn_data/bleve3d/npz/'
out_dir = '/home/jovyan/share/mgn_data/bleve3d/2d_Time5-55_Val'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
# Get all simulation case IDs that BLEVE height is 0.2
# So that z-axis can be removed without losing precision
df = pd.read_excel(file_path, engine='openpyxl')
df = df[df['Tank Height with Gas (m)'] == 0.2]
IDs = df['ID'].to_list()
IDs = [int(x[1:])+100000 if x[0] =='B' else int(x[1:])+200000  for x in IDs]
IDs = [x for x in IDs if ((x <= 100200) or ((x >= 200000) and (x <= 200200)))]
IDs = ['B' + str(int(str(x)[1:])) if x < 200000 else 'P' + str(int(str(x)[1:])) for x in IDs]
df_new = df[df['ID'].isin(IDs)]
df_new['volume'] = df_new['Tank Width (m)'] * df_new['Tank Length (m)'] * 0.4
print(df_new.describe())

# Read npz and save BLEVE 2D train.h5, valid.h5, test.h5

In [ ]:
import pandas as pd
import numpy as np
import h5py
import glob
import random
import os

file_path = '/home/jovyan/work/meshGraphNets_pytorch/data/bleve3d/BLEVE_Open_500Butan_500Propane_5-40m_8outputs.xlsx'
npz_dir = '/home/jovyan/share/mgn_data/bleve3d/npz/'
out_dir = '/home/jovyan/share/mgn_data/bleve3d/2d_Time5-55_Val'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
# Get all simulation case IDs that BLEVE height is 0.2
# So that z-axis can be removed without losing precision
df = pd.read_excel(file_path, engine='openpyxl')
df = df[df['Tank Height with Gas (m)'] == 0.2]
IDs = df['ID'].to_list()
IDs = [int(x[1:])+100000 if x[0] =='B' else int(x[1:])+200000  for x in IDs]
IDs = [x for x in IDs if ((x <= 100200) or ((x >= 200000) and (x <= 200200)))]
print(len(IDs))
npzs = [str(x) + '.npz' for x in IDs]
random.shuffle(npzs)

train_f = h5py.File(os.path.join(out_dir, "train.h5"), "a")
valid_f = h5py.File(os.path.join(out_dir, "valid.h5"), "a")
test_f = h5py.File(os.path.join(out_dir, "test.h5"), "a")

count_val, count_test = 0, 0
for idx, npz in enumerate(npzs):
    traj_name = npz.split('/')[-1].split('.')[0]
    data = np.load(npz_dir + npz)
    
    print(f'Processing {idx}/52, {traj_name}...')
    
    # Sampling data so that the domain is 20*20*10 meters and 35 timesteps of 70 ms
    pressure = data['pressure']
    grid = data['coordinate']
    pressure = pressure[5:55:2, 1, :, :].reshape(25, -1)
    grid = grid[1, :, :, :2].reshape(-1, 2)
    node_type = np.zeros((pressure.shape[1],))
    print('shape: ', pressure.shape, grid.shape, node_type.shape)
    print('Init pressure', pressure[0].max())
    if idx < 47:
        grp = train_f.create_group(traj_name)
        grp.create_dataset('pressure', data=pressure)
        grp.create_dataset('node_type', data=node_type)
        grp.create_dataset('grid', data=grid)
    if idx >= 47:
        print('added to val')
        count_val += 1
        grp = valid_f.create_group(traj_name)
        grp.create_dataset('pressure', data=pressure)
        grp.create_dataset('node_type', data=node_type)
        grp.create_dataset('grid', data=grid)
        print('added to test')
        count_test += 1
        grp = test_f.create_group(traj_name)
        grp.create_dataset('pressure', data=pressure)
        grp.create_dataset('node_type', data=node_type)
        grp.create_dataset('grid', data=grid)

print(count_test)
np.save(os.path.join(out_dir, 'grid'), grid)        
train_f.close()
valid_f.close()
test_f.close()
print('Finish')

In [ ]:
import numpy as np
import h5py
import os

# 100018, 100153, 100084, 200017, 200124,
# Open the file in read mode
with h5py.File('/home/jovyan/share/mgn_data/bleve3d/2d_Time5-35/valid.h5', 'r') as f:
    # Iterate over the groups
    for group_name in f:
        group = f[group_name]

        # Print the group name
        print(f'Group name: {group_name}')

        # Iterate over items in the group and print them
        for item_name in group:
            print(f'Item: {item_name}')
            print(f'Content: {group[item_name].shape}')  # Assumes the item is a dataset


# Misc

In [ ]:
# Customise learning rate schedule: Cosine with warmup

import matplotlib.pyplot as plt
import torch
from torchvision.models import resnet18
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts

model = resnet18()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = CosineAnnealingWarmupRestarts(optimizer,
                                              first_cycle_steps=100000,
                                              cycle_mult=1,
                                              max_lr=0.001,
                                              min_lr=0,
                                              warmup_steps=1000,
                                              gamma=0.1)

steps = 300000
lrs = []
for step in range(steps):
    lrs.append(scheduler.get_lr())
    scheduler.step()

plt.plot(lrs)
print(lrs[-1])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

file = '/home/jovyan/share/mgn_data/bleve3d/npz/100139.npz'

data = np.load(file)
timestep = 0
z_plane = 1

## Create the figure and axes
fig = plt.figure(figsize=(25, 5))

pressure = data['pressure']
grid = data['coordinate']

timesteps = 10
ps = []
for timestep in range(timesteps):
    p = pressure[timestep, z_plane]
    pmax = p.max()
    print(pmax)
    ps.append(pmax)
    mask = np.logical_and(p > -0.001, p < 0.001)
    p[mask] = np.nan
    
    ax = fig.add_subplot(1, timesteps, timestep+1)
    ax.set_xlim((-20, 20))
    ax.set_ylim((-20, 20))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    x = grid[z_plane, ..., 0].flatten()
    y = grid[z_plane, ..., 1].flatten()
    scatter = ax.scatter(x, y, c=p, cmap='bwr', vmin=-1, vmax=1, s=5, edgecolors='none')
    msg_time = f'Time: {timestep} ms'
    msg_p = 'Max P: {:.2f} bar'.format(pmax)
    ax.annotate(msg_time, xy=(0, 1), xycoords='axes fraction', xytext=(0.1, 0.2), textcoords='axes fraction', fontsize=13)
    ax.annotate(msg_p, xy=(0, 1), xycoords='axes fraction', xytext=(0.1, 0.1), textcoords='axes fraction', fontsize=13)

In [ ]:
import pandas as pd
import numpy as np
import h5py
import glob
import random
import os

file_path = '/home/jovyan/work/meshGraphNets_pytorch/data/bleve3d/BLEVE_Open_500Butan_500Propane_5-40m_8outputs.xlsx'
npz_dir = '/home/jovyan/share/mgn_data/bleve3d/npz/'
out_dir = '/home/jovyan/share/mgn_data/bleve3d/2d_Time5-55_Val'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
# Get all simulation case IDs that BLEVE height is 0.2
# So that z-axis can be removed without losing precision
df = pd.read_excel(file_path, engine='openpyxl')
df = df[df['Tank Height with Gas (m)'] == 0.2]
print(df.shape)

In [ ]:
import pandas as pd
import numpy as np
import h5py
import glob
import random
import os

npz_dir = '/home/jovyan/share/mgn_data/bleve3d/npz/100018.npz'
out_dir = '/home/jovyan/share/mgn_data/bleve3d/'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
test_f = h5py.File(os.path.join(out_dir, "test_newGeometry.h5"), "w")

traj_name = '100018'
data = np.load(npz_dir)

# Sampling data so that the domain is 20*20*10 meters and 35 timesteps of 70 ms
pressure = data['pressure']
grid = data['coordinate']
pressure = pressure[5:55:2, 1, ::2, ::2].reshape(25, -1)
grid = grid[1, ::2, ::2, :2].reshape(-1, 2)
node_type = np.zeros((pressure.shape[1],))
print('shape: ', pressure.shape, grid.shape, node_type.shape)
print('Init pressure', pressure[0].max())

grp = test_f.create_group(traj_name)
grp.create_dataset('pressure', data=pressure)
grp.create_dataset('node_type', data=node_type)
grp.create_dataset('grid', data=grid)

np.save(os.path.join(out_dir, 'grid'), grid)        
test_f.close()
print('Finish')

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(grid[:,0], grid[:,1], c=pressure[10, :]*100, cmap='bwr', vmin=-10, vmax=10, s=30, edgecolors='none')